In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date

In [0]:
df_consumos = (
    spark.read.table("dev.transient.consumos")
    .select(
        F.col("consumo_id"),
        F.col("hospede_id"),
        F.col("quarto_id"),
        F.col("data_consumo"),
        F.col("tipo_servico"),
        F.col("descricao"),
        F.col("valor"),  
        F.col("pago"),
    )
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
catalog_table = "dev.raw.tb_consumos"

try:
    delta_table = DeltaTable.forName(spark, catalog_table)

    delta_table.alias("target").merge(
        df_consumos.alias("source"),
        "target.reserva_id = source.reserva_id"
    ).whenMatchedUpdate(
        set={
            **{col: f"source.{col}" for col in df_consumos.columns if col != "update_date"},
            "update_date": current_date()
        }
    ).whenNotMatchedInsertAll().execute()

except Exception:
    df_consumos.write.format("delta").mode("overwrite").saveAsTable(catalog_table)

